In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


In [2]:
import os

# Check if img folder exists and list its contents
if os.path.exists('img'):
    print("Contents of img folder:")
    for item in os.listdir('img'):
        item_path = os.path.join('img', item)
        if os.path.isfile(item_path):
            print(f"  📄 {item}")
        elif os.path.isdir(item_path):
            print(f"  📁 {item}/")
            # Print contents of subdirectory
            print(f"    Contents of {item}/:")
            try:
                for subitem in os.listdir(item_path):
                    subitem_path = os.path.join(item_path, subitem)
                    if os.path.isfile(subitem_path):
                        print(f"      📄 {subitem}")
                    elif os.path.isdir(subitem_path):
                        print(f"      📁 {subitem}/")
            except PermissionError:
                print(f"      ❌ Permission denied")
    
    # Show total number of items
    total_items = len(os.listdir('img'))
    print(f"\nTotal items: {total_items}")
else:
    print("img folder does not exist")



Contents of img folder:
  📁 test/
    Contents of test/:
      📁 daisy/
      📁 dandelion/
  📁 train/
    Contents of train/:
      📁 daisy/
      📁 dandelion/
  📁 valid/
    Contents of valid/:
      📁 daisy/
      📁 dandelion/

Total items: 3


In [3]:
# Count daisy and dandelion images in each folder
if os.path.exists('img'):
    for folder_name in os.listdir('img'):
        folder_path = os.path.join('img', folder_name)
        if os.path.isdir(folder_path):
            print(f"\n📁 {folder_name}:")
            
            daisy_count = 0
            dandelion_count = 0
            
            try:
                # Check if there are daisy and dandelion subfolders
                subfolders = [item for item in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, item))]
                
                for subfolder in subfolders:
                    subfolder_lower = subfolder.lower()
                    subfolder_path = os.path.join(folder_path, subfolder)
                    
                    if 'daisy' in subfolder_lower:
                        try:
                            daisy_files = [f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))]
                            daisy_count = len(daisy_files)
                            print(f"  🌼 Daisy folder: {daisy_count} images")
                        except PermissionError:
                            print(f"  🌼 Daisy folder: ❌ Permission denied")
                    
                    elif 'dandelion' in subfolder_lower:
                        try:
                            dandelion_files = [f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))]
                            dandelion_count = len(dandelion_files)
                            print(f"  🌻 Dandelion folder: {dandelion_count} images")
                        except PermissionError:
                            print(f"  🌻 Dandelion folder: ❌ Permission denied")
                
                if daisy_count == 0 and dandelion_count == 0:
                    print(f"  ⚠️  No daisy or dandelion subfolders found")
                else:
                    print(f"  📊 Total: {daisy_count + dandelion_count}")
                
            except PermissionError:
                print(f"  ❌ Permission denied to access {folder_name}")
    
    # Calculate overall totals
    print(f"\n{'='*40}")
    print("OVERALL SUMMARY:")
    total_daisy = 0
    total_dandelion = 0
    
    for folder_name in os.listdir('img'):
        folder_path = os.path.join('img', folder_name)
        if os.path.isdir(folder_path):
            try:
                subfolders = [item for item in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, item))]
                
                for subfolder in subfolders:
                    subfolder_lower = subfolder.lower()
                    subfolder_path = os.path.join(folder_path, subfolder)
                    
                    if 'daisy' in subfolder_lower:
                        try:
                            daisy_files = [f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))]
                            total_daisy += len(daisy_files)
                        except PermissionError:
                            continue
                    
                    elif 'dandelion' in subfolder_lower:
                        try:
                            dandelion_files = [f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))]
                            total_dandelion += len(dandelion_files)
                        except PermissionError:
                            continue
                            
            except PermissionError:
                continue
    
    print(f"🌼 Total Daisy images: {total_daisy}")
    print(f"🌻 Total Dandelion images: {total_dandelion}")
    print(f"📊 Grand Total: {total_daisy + total_dandelion}")
else:
    print("img folder does not exist - cannot count daisy and dandelion images")




📁 test:
  🌼 Daisy folder: 77 images
  🌻 Dandelion folder: 105 images
  📊 Total: 182

📁 train:
  🌼 Daisy folder: 529 images
  🌻 Dandelion folder: 746 images
  📊 Total: 1275

📁 valid:
  🌼 Daisy folder: 163 images
  🌻 Dandelion folder: 201 images
  📊 Total: 364

OVERALL SUMMARY:
🌼 Total Daisy images: 769
🌻 Total Dandelion images: 1052
📊 Grand Total: 1821


In [4]:
# Define directory paths
TRAIN_DIR = 'img/train'
TEST_DIR = 'img/test'
VAL_DIR = 'img/valid'



In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(224, 224),
    shuffle=True,
    class_mode = 'binary')

val_generator = datagen.flow_from_directory(
    VAL_DIR,
    target_size=(224, 224),
    shuffle=False,
    class_mode = 'binary')

test_generator = datagen.flow_from_directory(
    TEST_DIR,
    target_size=(224, 224),
    shuffle=False,
    class_mode = 'binary')


Found 1275 images belonging to 2 classes.
Found 364 images belonging to 2 classes.
Found 182 images belonging to 2 classes.


In [6]:
from tensorflow.keras.applications import MobileNetV2
import keras_tuner as kt

base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

In [7]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > 0.95:
            print("\nReached 95% accuracy so cancelling training!")
            self.model.stop_training = True

In [8]:
def build_model(hp):
    model = tf.keras.models.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(hp.Float('dropout', 0.2, 0.8, step=0.1)),
        tf.keras.layers.Dense(hp.Int('units', 32, 512, step=32), activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='my_dir',
    project_name='mobilenet_tuning'
)


In [9]:
# Actually perform the hyperparameter search and training
tuner.search(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[MyCallback()]
)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

Trial 10 Complete [00h 01m 19s]
val_accuracy: 0.44780218601226807

Best val_accuracy So Far: 0.9148351550102234
Total elapsed time: 00h 33m 32s



In [12]:
test_loss, test_acc = best_model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")

6/6 [==============================] - 9s 806ms/step - loss: 0.2613 - accuracy: 0.9011
Test accuracy: 0.901098906993866
Test loss: 0.26126617193222046


In [13]:
# Save the best model
best_model.save('best_model.h5')

c:\Users\Swift GO 14\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
# Show the best trial results
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: {best_hps.values}")

# Show results summary
tuner.results_summary()

Best hyperparameters: {'dropout': 0.6000000000000001, 'units': 96, 'learning_rate': 0.00014824655672408831}
Results summary
Results in my_dir\mobilenet_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 00 summary
Hyperparameters:
dropout: 0.6000000000000001
units: 96
learning_rate: 0.00014824655672408831
Score: 0.9148351550102234

Trial 01 summary
Hyperparameters:
dropout: 0.6000000000000001
units: 320
learning_rate: 0.0031256760850773423
Score: 0.44780218601226807

Trial 02 summary
Hyperparameters:
dropout: 0.30000000000000004
units: 384
learning_rate: 0.0029594770617025457
Score: 0.44780218601226807

Trial 03 summary
Hyperparameters:
dropout: 0.30000000000000004
units: 64
learning_rate: 0.00431019016344334
Score: 0.44780218601226807

Trial 04 summary
Hyperparameters:
dropout: 0.30000000000000004
units: 256
learning_rate: 0.0002151735195022257
Score: 0.44780218601226807

Trial 05 summary
Hyperparameters:
dropout: 0.7
units: 384
learning_rate: 0.00101